GitHub 
https://github.com/rinnakk/japanese-pretrained-models   
https://huggingface.co/rinna/japanese-gpt-1b  

<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/rinna_japanese_gpt2_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ランタイムの設定
「ランタイム」→「ランタイムのタイプを変更」→「ハードウェアアクセラレータ」をGPUに変更

# 実行方法
「ランタイム」→「すべてのセルを実行」を選択

# ライブラリインストール

In [ ]:
# Huggingface Transformers
!pip install transformers==4.16.0
# Sentencepiece
!pip install sentencepiece==0.1.96

# トークナイザーとモデルのロード

In [ ]:
import torch
from transformers import T5Tokenizer, AutoModelForCausalLM
import re

# トークナイザーとモデルのロード
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt-1b")
model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt-1b")

# GPU使用
if torch.cuda.is_available():
    model = model.to("cuda")

# 自問自答を定義

In [ ]:
#@markdown ## 独り言回数
round_trip = 20 #@param {type:"slider", min:0, max:100, step:1}

#@markdown ## 独り言のトピック
topic = "\u4EBA\u9593\u3068\u4EBA\u5DE5\u77E5\u80FD\u306F\u3069\u3046\u306A\u3063\u3066\u3044\u304F\u306E\u3060\u308D\u3046\u304B\u3002" #@param {type:"string"}
text = "りんなちゃんの独り言。りんなちゃん:「おはよう私」りんなちゃん:「" + topic + "」りんなちゃん:「"

#@markdown ### parameter変更(Option)
#@markdown 次のトークン確率をモジュール化するために使用される値
temperature = 1 #@param {type:"slider", min:0.0, max:1.0, step:0.1}
#@markdown 繰り返しペナルティのパラメータ。1.0はペナルティなし
repetition_penalty = 1 #@param {type:"slider", min:0.0, max:1.0, step:0.1}
#@markdown 長さに対する指数関数的なペナルティ。1.0はペナルティなし
length_penalty = 1.0 #@param {type:"slider", min:0.0, max:1.0, step:0.1}

In [ ]:
print("りんなちゃんの独り言。topic:", topic)

pos = 3 # 括弧の取得位置
for round_num in range(round_trip):
  token_ids = tokenizer.encode(text, add_special_tokens=False, return_tensors="pt")
  max_length = 100
  if max_length < len(text):
    max_length = len(text) + 30

  # りんなちゃんのテキスト生成
  with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        max_length=max_length,
        min_length=50,
        do_sample=True,
        top_k=500,
        top_p=0.95,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        bad_word_ids=[[tokenizer.unk_token_id]],
        temperature = temperature,
        repetition_penalty = repetition_penalty,
        length_penalty = length_penalty
        )
  output = tokenizer.decode(output_ids.tolist()[0])
  # 半角を全角に正規化
  output = output.translate(str.maketrans({chr(0xFF01 + i): chr(0x21 + i) for i in range(94)}))

  # りんなちゃんの先頭の独り言のみ取得
  prefix = "りんなちゃん:「"
  suffix = "」"
  pre = output.split(prefix)
  post = pre[pos].split(suffix)

  # 」で閉じずにりんなちゃんが次の独り言を続けた場合に対処
  if "りんなちゃん:" in post[0]:
    post[0] = post[0].split("りんなちゃん:")[0]
  # 」で閉じずに終了した場合
  if "</s>" in post[0]:
    post[0] = post[0].replace("</s>", "")

  print("独り言", str(round_num+1), ":", post[0])
  
  # textに付加
  text += post[0] + "りんなちゃん:「"

  # 次回取得位置更新
  pos += 1
